#### Jérémy TREMBLAY

# TP4 : Upper Confidence Bound

In [1]:
# Import the libraries that will be used in this notebook.
import numpy as np
from abc import ABC, abstractmethod

# Import the pyplot module from matplotlib with the plt alias.
import matplotlib.pyplot as plt

# Other usefull libraries
import math

## Task 1: Create a `Bandit` class

**Consigne :** Créer une représentation d’une machine bandit dans une classe nommée `Bandit` :  
* Qui prend en paramètre la moyenne et l’écart-type d’une distribution gaussienne.
* Définir la méthode `draw` qui permet de tirer le bras du bandit et de retourner un gain potentiel.  

Ce gain sera directement fournit par la distribution et ses paramètres.

Let's define the class.

In [2]:
class Bandit:
    """Bandit class. Represents a bandit in a Casino game.
    """

    def __init__(self, mean, std_dev):
        """
        Bandit constructor.

        Args:
            mean: {float} -- The mean of the Gaussian distribution.
            std_dev: {float} -- The standart deviation of the Gaussian distribution.
        """
        self.mean = mean
        self.std_dev = std_dev
    
    def draw(self):
        """
        Simulate pulling the bandit's arm and return a potential gain.

        Returns:
            {float} -- The potential gani drawn from the Gaussian distribution.
        """
        return np.random.normal(self.mean, self.std_dev)

Let's create a Bandit and test it.

In [3]:
bandit_test = Bandit(20, 40)
print(bandit_test.draw())
print(bandit_test.draw())
print(bandit_test.draw())

-39.8659070721706
10.032084283078067
-23.387847901037027


Ok it seems coherent.

## Task 2: Create a `Casino` class

**Consigne :** Créer une représentation d’un casino, à partir d’une classe `Casino` :  
* Un casino est composée de plusieurs machines `bandits`.
* Définir la méthode `play` qui permet de tirer le bras d’un bandit ciblé par son index et retourne son gain si le bandit est bien disponible.
* Créer une méthode de type *property*, qui permet de retourner le nombre de `bandits` présents dans le casino.

In [4]:
class Casino:
    """Casino class, represents a Casino where players can play games (and lose money).
    """

    def __init__(self):
        """
        Casino constructor.

        Initializes an empty lsit to store bandits.
        """
        self.bandits = []

    def __init__(self, bandits):
        """
        Casino constructor.

        Initializes a list to store bandits.
        """
        self.bandits = []
        [self.bandits.append(bandit) for bandit in bandits]

    def play(self, index):
        """
        Simulate playing a specific bandit in the casino.

        Args:
            bandit_index: {int} -- Th index of the bandit to be played.

        Returns:
            {float} -- The gain obtained from playng the selected bandit.
        """
        if index >= 0 and index < len(self.bandits):
            return self.bandits[index].draw()
        return None
    
    @property
    def number_of_bandits(self):
        """
        Get the number of bandits in the casino.

        Returns:
            {int} -- The number of bandits in the casino.
        """
        return len(self.bandits)

Let's test the Casino.

In [5]:
casino_test = Casino([bandit_test])
print(casino_test.play(-1))
print(casino_test.play(0))
print(casino_test.play(1))

None
31.051435491285524
None


## Task 3: Create an `Agent` class

**Consigne :** Créer une classe abstraite `Agent` telle que :
* Elle est associée à une instance de casino.
* Enregistre les gains obtenus pour chaque machine bandit présent dans le casino.
* Permet de connaître le nombre de tour de jeu effectué.
* Permet l’accès à une propriété `rewards`, qui fournit la somme des récompenses de l’agent.
* Une méthode abstraite privée `_policy` qui détermine le bandit sélectionné pour un tour de jeu (politique de choix). Elle retourne l’index de ce bandit.
* Une méthode `select` générique qui permet d’activer la politique de choix du bandit, de jouer ce bandit et de mettre à jour les récompenses reçues pour ce bandit.
* Une méthode de surcharge `__str__` qui permet un affichage simple de la classe : type actuel de la classe, informations sur les bandits et gain total (vous pouvez revenir sur son développement par la suite).

In [6]:
class Agent(ABC):
    """Agent class. Represents a worker that make some manipulations in the casino.
    """

    def __init__(self, casino):
        """
        Agent constructor.

        Args:
            casino: {Casino} -- The Casino object associated with the agent.
        """
        self.casino = casino
        self.gains = [0 for _ in range(casino.number_of_bandits)]
        self.number_of_turns = 0

    @abstractmethod
    def _policy(self):
        """
        Abstract method to determine the bandit selected for a round of play.

        Returns:
            {int} -- The index of the selected bandit.
        """
        pass

    def select(self):
        """
        Activate the bandit selection policy, play the selected bandit, and update rewarsd.

        Returns:
            {float} -- The gain obtained from playing the selected bandit.
        """
        bandit_index = self._policy()
        reward = self.casino.play(bandit_index)
        self.gains[bandit_index] += reward
        self.number_of_turns += 1
        return reward

    @property
    def rewards(self):
        """
        Get the number of rewards at the total for the casino.

        Returns:
            {int} -- The gains made in the casino by all the bandits.
        """
        return sum(self.gains)

    def __str__(self):
        """
        String representation of the Agent class.

        Returns:
            {str} -- The string representation of the Agent class.
        """
        string = f"{type(self).__name__}:\n"
        for i in range(self.casino.number_of_bandits):
            string += f"- {i}: {self.gains[i]:0.3f}\n"
        return f"{string}Total: {self.rewards} in {self.number_of_turns} rounds."

## Task 4: Create a greedy agent

**Consigne :** À partir de cette représentation abstraite de votre agent, créer l’agent `Greedy` (Glouton), qui aura comme politique de choix d’action :  
* Tester une fois chaque machine présente dans le casino.
* Puis, de toujours sélectionner celle qui lui avait fournie le meilleur gain.

In [7]:
class Greedy(Agent):
    def __init(self, casino):
        """
        GreedyAgent constructor.

        Initializes the GreedyAgent with the casino.

        Args:
            casino: {Casino} -- The Casino object associated with the agent.
        """
        super().__init__(casino)

    def _policy(self):
        """
        Select the bandit based on the greedy policy (always the one who produced the best score once a turn for each).

        Returns:
            {int} -- The index of the selected bandit.
        """
        if self.number_of_turns < self.casino.number_of_bandits:
            return self.number_of_turns
        else:
            return np.argmax(self.gains)

Let's test the class.

In [8]:
# Create a Casino with three bandits and the agent.
casino = Casino([Bandit(mean=10, std_dev=1), Bandit(mean=5, std_dev=2), Bandit(mean=2, std_dev=0.5)])
greedy_agent = Greedy(casino)

# Make the agent play for a certain number of rounds.
num_rounds_to_play = 1000
for _ in range(num_rounds_to_play):
    greedy_agent.select()

# Print the final results.
print(greedy_agent)

Greedy:
- 0: 9972.260
- 1: 1.268
- 2: 2.293
Total: 9975.821706919874 in 1000 rounds.


## Task 5: Create an epsilon greedy agent

**Consigne :** Créer maintenant l’agent `EpsilonGreedy`, qui aura comme politique de choix d’action :
* Tester une fois chaque machine présente dans le casino.
* Puis, de toujours sélectionner celle qui lui avait fournie le meilleur gain sauf sous contrainte d’un critère probabiliste *ϵ* € [0, 1], de choisir une action aléatoire.

In [9]:
class EpsilonGreedy(Agent):
    def __init__(self, casino, epsilon):
        """
        EpsilonGreedy constructor.

        Initializes the EpsilonGreedy agetn with the casino and epsilon.

        Args:
            casino: {Casino} -- The Casino object associated with the agent.
            epsilon: {float} -- The exploration-exploitation trade-off parameter.
        """
        super().__init__(casino)
        self.epsilon = epsilon

    def _policy(self):
        """
        Select the bandit based on the epsilon-greedy policy.

        Returns:
            {int} -- The index of the selected bandit.
        """
        if self.number_of_turns < self.casino.number_of_bandits:
            return self.number_of_turns
        else:
            if np.random.rand() < self.epsilon:
                # Explore: Choose a random bandit.
                bandit_index = np.random.randint(self.casino.number_of_bandits)
            else:
                # Exploit: Choose the bandit with the highest total rewards.
                bandit_index = np.argmax(self.rewards)

        return bandit_index

Let's test it.

In [10]:
# Create a casino with bandits.
bandit1 = Bandit(1.0, 0.1)
bandit2 = Bandit(2.0, 0.5)
bandit3 = Bandit(0.5, 0.2)

casino = Casino([bandit1, bandit2, bandit3])

# Create an EpsilonGreedy agent with an epsilon value of 0.1.
epsilon_greedy_agent = EpsilonGreedy(casino, epsilon=0.1)

# Run the agent.
for _ in range(1000):
    epsilon_greedy_agent.select()

# Display results.
print(epsilon_greedy_agent)

EpsilonGreedy:
- 0: 927.305
- 1: 80.120
- 2: 16.565
Total: 1023.9907817765684 in 1000 rounds.


We can see heere that it has sometimes test another bandit than the one that is best suitable for him, because of the epsilon parameter. More we increase it, more we can observe an average for the bandits used.

## Task 6: Define a casino context and simulate agents

**Consigne :** Définir un contexte de casino afin de simuler vos stratégies d’agents :
* Définir 4 bandits ayant respectivement les distributions suivantes : N(2, 3), N(-1, 6), N(-2, 3), N(5, 3).
* Simuler pour chaque action les gains potentiels obtenus par vos agents sur 1000 tours (nombre de sélections d’action).
* Analysez les résultats obtenues en faisant varier le paramètre *ϵ* de votre agent de type `EpsilonGreedy`.


In [11]:
# First let's define the bandits.
bandit1 = Bandit(2, 3)
bandit2 = Bandit(-1, 6)
bandit3 = Bandit(-2, 3)
bandit4 = Bandit(5, 3)

# Create the casino and the agent.
casino = Casino([bandit1, bandit2, bandit3, bandit4])
greedy_agent = Greedy(casino)
total_turns = 1000

# Let's simulate the agent.
for _ in range(total_turns):
    greedy_agent.select()

# Display Greedy agent's results.
print("Results for Greedy agent:")
print(greedy_agent)

# Simulate strategies for tje EpsilonGreedy greedy agent with different values for epsilon.
epsilon_values = [0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 1]

for epsilon in epsilon_values:
    epsilon_greedy_agent = EpsilonGreedy(casino, epsilon)
    for _ in range(total_turns):
        epsilon_greedy_agent.select()

    # Display EpsilonGreedy agent's results.
    print(f"\nResults for EpsilonGreedy agent with epsilon={epsilon}:")
    print(epsilon_greedy_agent)

Results for Greedy agent:
Greedy:
- 0: 0.547
- 1: -10.478
- 2: -3.496
- 3: 4933.841
Total: 4920.412933976326 in 1000 rounds.

Results for EpsilonGreedy agent with epsilon=0.1:
EpsilonGreedy:
- 0: 1551.618
- 1: -46.660
- 2: -85.496
- 3: 144.402
Total: 1563.865059954677 in 1000 rounds.

Results for EpsilonGreedy agent with epsilon=0.2:
EpsilonGreedy:
- 0: 1782.916
- 1: -33.997
- 2: -78.843
- 3: 229.742
Total: 1899.8178287643966 in 1000 rounds.

Results for EpsilonGreedy agent with epsilon=0.4:
EpsilonGreedy:
- 0: 1337.505
- 1: -24.146
- 2: -171.253
- 3: 519.266
Total: 1661.3730264593119 in 1000 rounds.

Results for EpsilonGreedy agent with epsilon=0.5:
EpsilonGreedy:
- 0: 1321.879
- 1: -119.012
- 2: -271.498
- 3: 560.954
Total: 1492.3229348004961 in 1000 rounds.

Results for EpsilonGreedy agent with epsilon=0.6:
EpsilonGreedy:
- 0: 989.963
- 1: -59.126
- 2: -282.208
- 3: 867.384
Total: 1516.013354117556 in 1000 rounds.

Results for EpsilonGreedy agent with epsilon=0.8:
EpsilonGreedy:
- 0

We can see that the Greedy agent consistently favors the bandit with the highest total reward, leading to significant gains from the bandit with mean 5 and standard deviation 3.

On the other hand, the EpsilonGreedy agent explores different bandits based on the epsilon parameter, balancing exploration and exploitatino. As epsilon increases, the agent becomes more explorative, resulting in a trade-off between discovering potentially better bandits and exploiting the currently best knonw bandit. This trade-off is reflected in the total rewards, with higher epsilon values leading to more exploration and potentially lower overall gains.

In summary, adjusting the epsilon parameter allows fine-tuning the exploration-exploitation strategy of the agent. Higher epsilon values promote exploration, providing a balance between exploiting the best-known bandit and discovering potentially better alternatives.

## Task 7: Create an UCB agent

**Consigne :** La formule de l'UCB est la suivante :

`UCB = Ri + c x sqrt(log(N) / ni)`

Développer l’agent UCB afin de le comparer comme précédemment, tel que :
* Comme pour les autres stratégies, chaque machine est explorée une fois.
* Puis, la stratégie de choix des actions (politique) est maintenant basée sur la formule UCB.
* L’action finalement sélectionnée est celle fournissant la valeur maximale UCB.

In [12]:
class UCB_Agent(Agent):
    """UCB_Agent class. Represents an agent using the UCB strategy.
    """

    def __init__(self, casino, c):
        """
        UCB_Agent constructor.

        Args:
            casino: {Casino} -- The Casino object associated with the agent.
            c: {float} -- The exploration-exploitation trade-off parameters.
        """
        super().__init__(casino)
        self.c = c
        self.total_plays = [1 for _ in range(casino.number_of_bandits)]

    def _policy(self):
        """
        Determine the bandit selected for a round of play using UCB.

        Returns:
            {int} -- The index of the selected bandt.
        """
        # Compute the "best" agent depending on the average win value and the number of time the bandit was selected.
        # Based on the UCB formula.
        ucb_values = [
            self.gains[i] / self.total_plays[i] +
            self.c * math.sqrt(math.log(max(1, self.number_of_turns)) / max(1, self.total_plays[i]))
            for i in range(self.casino.number_of_bandits)
        ]
        return ucb_values.index(max(ucb_values))

    def select(self):
        """
        Activate the UCB bandit selection policy, play the selected bandit, and update rewards.

        Returns:
            {float} -- The gain obtained from playing the selected bandit.
        """
        bandit_index = self._policy()
        reward = super().select()              # Call the base method.
        self.total_plays[bandit_index] += 1    # Edit to collect the number of time the bandit was pressed for each bandit.
        return reward

Let's test it now.

In [15]:
# Create a casino with bandits.
bandit1 = Bandit(2, 3)
bandit2 = Bandit(-1, 6)
bandit3 = Bandit(-2, 3)
bandit4 = Bandit(5, 3)

casino = Casino([bandit1, bandit2, bandit3, bandit4])
total_turns = 1000
results = []
c_values = [x for x in range(1, 20, 1)]

# Simulate actions for c values.
for c in c_values:
    agent_ucb = UCB_Agent(casino, c)
    
    # Play the bandits for a certain number of rounds.
    for _ in range(total_turns):
        agent_ucb.select()
    
    # Store the results.
    results.append(agent_ucb.rewards)

# Print the results.
for i, c in enumerate(c_values):
    print(f"UCB Agent with c={c}: Total Rewards = {results[i]}")

UCB Agent with c=1: Total Rewards = 4889.795662644114
UCB Agent with c=2: Total Rewards = 4903.450085576625
UCB Agent with c=3: Total Rewards = 4769.009455779889
UCB Agent with c=4: Total Rewards = 4791.858126119577
UCB Agent with c=5: Total Rewards = 4915.879716832812
UCB Agent with c=6: Total Rewards = 4752.485164404493
UCB Agent with c=7: Total Rewards = 4762.659381722687
UCB Agent with c=8: Total Rewards = 4858.622923490334
UCB Agent with c=9: Total Rewards = 4789.880436342095
UCB Agent with c=10: Total Rewards = 4559.001273767392
UCB Agent with c=11: Total Rewards = 4773.0366545064335
UCB Agent with c=12: Total Rewards = 4718.054482629464
UCB Agent with c=13: Total Rewards = 4667.231137189211
UCB Agent with c=14: Total Rewards = 4384.900214110032
UCB Agent with c=15: Total Rewards = 4328.0954355154845
UCB Agent with c=16: Total Rewards = 4529.02078019051
UCB Agent with c=17: Total Rewards = 4264.075427651087
UCB Agent with c=18: Total Rewards = 4085.8380587541037
UCB Agent with c=

**Conclusion:** the UCB Agent's total rewards demonstrate a varyng pattern based on different values of the exploration-exploitation trade-off parameter (c). While increasing values of c generally lead to higher total rewards, there is a point where further exploration negatively impacts overall performance. The optimal value for c appears to be around 5, where the agent achieves the highest total rewards. Beyond this point, the agent's performance tends to plateau or decrease.

## Questions

1. Quels sont les défauts des strategies dites gloutonnes ?
2. Quel est l’avantage de l’algorithme UCB ?
3. Quel est l’impact du paramètre c dans la formule d’estimation de valeur d’une action ?

1. **Downsides of Greedy Strategies:**
   - **Overlooking Potential Gems:** Greedy strategies tend to stick with what seems best at the moment without exploring other options. This might cause us to miss out on actions that could be more profitable in the long run.
   - **Stuck in the Comfort Zone:** Imagine if our strategy only focus on what seems locally optimal. We might get stuck in a comfortable routine and miss out on discovering better solutions.  

2. **Why UCB Rocks:**
   - **Finding the Sweet Spot:** The UCB algorithm strikes a cool balance between trying out new things and sticking with what has worked before. It considers how uncertain we re about each action, so we keep exploring while still favoring actions that have proven promising.  

3. **What’s up with the 'c' in UCB:**
   - **Dance Between Exploration and Exploitation :** The 'c' parameter in the UCB formula is like a dance partner, it decides how much we want to try new moves versus stikcing with the ones we know. A higher 'c' means we're more adventurous, but too high might lead to a messy dance. Finding the right 'c' is like adjusting the rhythm for the perfect balance between trying new things and sticking to what we know. Thi is the parameter that controls the trade-off between exploration and exploitation. The optimal choice of c depends on the problem and needs to be adjusted experimentally to find the right balance between exploration and exploitation, like we have do just before with our loop.